In [49]:
import numpy as np
import pandas as pd


In [50]:
df=pd.read_csv('diabetes.csv')

In [51]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [52]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [53]:
X=df.iloc[:,0:-1].values
y=df.iloc[:,-1].values

In [54]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X=sc.fit_transform(X)

In [55]:
X.shape

(768, 8)

In [56]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [57]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [58]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))

model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [59]:
model.fit(X_train,y_train,batch_size=32,epochs=10,validation_data=(X_test,y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5786 - loss: 0.6866 - val_accuracy: 0.7143 - val_loss: 0.6178
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6773 - loss: 0.6402 - val_accuracy: 0.7727 - val_loss: 0.5679
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6878 - loss: 0.6028 - val_accuracy: 0.7727 - val_loss: 0.5336
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7358 - loss: 0.5581 - val_accuracy: 0.7727 - val_loss: 0.5077
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7360 - loss: 0.5393 - val_accuracy: 0.7792 - val_loss: 0.4890
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7338 - loss: 0.5407 - val_accuracy: 0.7922 - val_loss: 0.4761
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7776 - loss: 0.5093 - val_accuracy: 0.8052 - val_loss: 0.4647
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7795 - loss: 0.4916 - val_accuracy: 0.8182 - val_loss

In [60]:
import kerastuner as kt

In [61]:
def build_model(hp):
  model=Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [62]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [63]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [64]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [65]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [66]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=11,validation_data=(X_test,y_test))

Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.7158 - loss: 0.5855 - val_accuracy: 0.7857 - val_loss: 0.5430
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7078 - loss: 0.5718 - val_accuracy: 0.7792 - val_loss: 0.5192
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7418 - loss: 0.5411 - val_accuracy: 0.8052 - val_loss: 0.5004
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7531 - loss: 0.5166 - val_accuracy: 0.7987 - val_loss: 0.4864
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7409 - loss: 0.5265 - val_accuracy: 0.8117 - val_loss: 0.4752
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7611 - loss: 0.4943 - val_accuracy: 0.7922 - val_loss: 0.4638
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7460 - loss: 0.5065 - val_accuracy: 0.7987 - val_loss: 0.4564
Epoch 19/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7584 - loss: 0.4852 - val_accuracy: 0

In [73]:
def build_model(hp):
    model = Sequential()
    # Get the best hyperparameter values from the tuner's oracle
    best_hp = tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters
    units = best_hp.get('units')  # Get the 'units' value from the best trial
    model.add(Dense(units=units, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Assuming 'tuner' is already defined and search has been performed:


In [68]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [69]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [74]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam', 'units': 8}

In [76]:
model = tuner.get_best_models(num_models=1)[0]

ValueError: A total of 2 objects could not be loaded. Example error message for object <Dense name=dense, built=True>:

The shape of the target variable and the shape of the target value in `variable.assign(value)` must match. variable.shape=(8, 8), Received: value.shape=(8, 32). Target variable: <KerasVariable shape=(8, 8), dtype=float32, path=sequential/dense/kernel>

List of objects that could not be loaded:
[<Dense name=dense, built=True>, <Dense name=dense_1, built=True>]

In [77]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(72,activation='relu',input_dim=8))
    # Get the best hyperparameter values from the tuner's oracle
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        model.add(Dense(72,activation='relu',input_dim=8))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [78]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [79]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [80]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [81]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.8114 - loss: 0.4081 - val_accuracy: 0.8052 - val_loss: 0.4485
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7975 - loss: 0.4273 - val_accuracy: 0.8117 - val_loss: 0.4485
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8125 - loss: 0.4196 - val_accuracy: 0.8117 - val_loss: 0.4493
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8026 - loss: 0.4213 - val_accuracy: 0.8117 - val_loss: 0.4513
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8242 - loss: 0.4016 - val_accuracy: 0.8052 - val_loss: 0.4500
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8121 - loss: 0.3946 - val_accuracy: 0.8052 - val_loss: 0.4502
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8087 - loss: 0.4154 - val_accuracy: 0.8052 - val_loss: 0.4503
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7861 - loss: 0.4189 - val_accuracy: 0.811